# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [17]:
from os import walk, getcwd
from ticktack.fitting import SingleFitter
from ticktack import load_presaved_model
from math import exp

In [3]:
models = { # This dictionary contains the units for the fluxes and production function
    "Guttler14": {  # Units of the Guttler 2014 paper
        "production_rate_units": "atoms/cm^2/s",    # Units of the production rate 
        "flow_rate_units": "Gt/yr"                  # Units of the fluxes
    }
}

In [6]:
data_sets = []  # List for storing the data set locations
data_sets_directory = f"{getcwd()}/datasets/775AD" # Home directory of the data 
for (root, dirs, files) in walk(data_sets_directory):    # Looping over directories 
    for file in files:  # Looping through the files 
        file_path = root + "/" + file   # Setting up the path 
        data_sets.append(file_path)  # Extending the stored directoriess

In [ ]:
def production_function(time, *parameters):
    """
    Given the time and parameters of the production function. Generates the total production of carbon 14 at that time using a linear model and a super-gaussian pulse. 
    Parameters:
        time: float -> The time in years.
        parameters: tuple -> The parameters of the production function. In this case, steady state, long term trend, middle, duration and amplitude.
    Returns:
        float -> The carbon 14 production at time.
    """
    const, lin_coeff, center, duration, amplitude = parameters
    super_gaussian = amplitude * exp(((time - center) / (duration / 2)) ** 8)
    long_term_trend = const + lin_coeff * time
    return super_gaussian + long_term_trend

In [ ]:
def get_model(model: str, datum: str):
    """
    Using a `ticktack.SingleFitter` this function compiles the model and selects the appropriate production function.
    """
    cbm = load_presaved_model( # Generating the CarbobBoxModel using ticktack
        model,  # Name of the model as looped from the models dictionary 
        production_rate_units=models[model]["production_rate_units"], 
        flow_rate_units=models[model]["flow_rate_units"])

    bayesian_model = SingleFitter(cbm)   # Fitting a model 
    bayesian_model.prepare_function(production_function)    # Generating the simple sin model
    bayesian_model.load_data(datum)   # Loading the data into the model  

    return bayesian_model

In [ ]:
def get_production_function(model: SingleFitter):
    """
    Parameters:
        model: `str` - The `CarbonBoxModel` that is to be used
        data: `str` - The dataset that the production function is to be fitted to (.csv)
    Returns:
        production: `function` - The ideal production function 
    """
    # Return to nedler-mead 
    # Simulate 50 year "template/burn in"
    # Fit the miyake data with mcmc including sinusoid. 
    # Not too comfortable with atmospheric dynamics.
        # Stratospheric pull down ? radio carbon
        # Pulled down close to the surface at mid-year
    # ANU carbon models
        # Cameron O'neil
    # Sharp-rise (Late rise) 774 AD datasets

    # The template that I want to pick removes the sinuspoid but uses the event parameters that were fitted with the sinusoid. Using some measure like MLE or mean. That way we are isolating the event. Save this to a file and evaluate on a 50 year grid with the event happening in the middle. 

    # So split the file into get_template.ipynb -> Run once and save the output onto a 50 year grid. 
    initial_parameters = array([model.time_data[len(model.time_data) // 2], 1./12, pi/2., 81./12])
    # Guess the initial parameters by hand.

    # emcee is an implementation of affine invariant ensamble mcmc. Metropolis hastings is the vanilla that has one walker and each evaluation is a step/not step of the walker. You can also do parallel tempering (MH). Ensamble mcmc is something different, you take a hundred points and propose a dist based on the stretch move. The stretch move draws lines between the points and takes a step along the line (from some dist). This results in the thing being affine, which is a generalisation of linear. This is provably optimal for doing mcmc on a multivariate gaussian.

    # Need to initialise the parameters as a multi-dimensional gaussian noise, to stetch the lines. This is what the condition number means. The condition number is the ratio of the smallest and largest eigenvalue.

    # DFM's website demonstrates how to do this 
    sampler = emcee.EnsembleSampler(4, 4, model.log_likelihood)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(initial_parameters, 200, progress=True);

    print("Running production...")
    sampler.reset()
    sampler.run_mcmc(p0, model.simple_sinusoid, progress=True);
    samples = sampler.flatchain

    parameters = {  # A dictionary of the model parameters for the `simple_sinusoid`
        "Start Date (yr)": None,    # The year that the event began 
        "Duration (yr)": None,      # Number of years that the event occured over 
        "Phase (yr)": None,         # The phase shift of the sinusoidal production function 
        "Area": None               #? What are the units?
    } 

    for i, parameter in enumerate(parameters):  # Looping through the parameters 
        parameters[parameter] = {   # Nested dictionary to store statistical information
            "mean": mean(samples[:, i]),    # Storing the mean of the samples produced by mcmc
            "median": median(samples[:, i]),# Storing the median in addition to the mean 
            "variance": var(samples[:, i])  # Storing the variance of the parameter
        }
    parameters["Steady Production"] = model.steady_state_production   
    
    return parameters